p100 testing new loss2

In [6]:
import time
import numpy as np
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# from tqdm.notebook import tqdm

In [7]:
from FADAnet.FADAloader import *
from FADAnet.FADAmodule import *

In [8]:
# loss version: loss = loss1 + gamma*loss2/(n_support*90) + theta*loss3/(10*n_support)
n_epoch = 161  # total number of epoch
n_epoch_pt = 0
batch_size = 256
batch_size_test = 512
lr = 0.0007
gamma = 0.06
theta = 0.05
n_support = 7
loss3_margin = 0.7

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
np.random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed(777)

In [9]:
train_dataloader=mnist_dataloader(batch_size=batch_size,train=True)
test_dataloader=mnist_dataloader(train=False)
real_test_loader = svhn_dataloader(batch_size = batch_size_test, train=False)

classifier=Classifier()
encoder=Encoder()

classifier.to(device)
encoder.to(device)
loss_fn1 = torch.nn.CrossEntropyLoss()
loss_fn2 = torch.nn.CosineEmbeddingLoss()
loss_fn3 = torch.nn.CosineEmbeddingLoss(margin=loss3_margin)
optimizer=torch.optim.Adadelta(list(encoder.parameters())+list(classifier.parameters()))

X_t, Y_t = create_target_samples(n=n_support)

Using downloaded and verified file: ./data/SVHN/test_32x32.mat


In [10]:
# for epoch in tqdm(range(n_epoch)):
for epoch in range(n_epoch):
    
    for data,labels in train_dataloader:
        data=data.to(device)
        labels=labels.to(device)
        X_t = X_t.to(device)
        Y_t = Y_t.to(device)
        optimizer.zero_grad()
        
        map_s = encoder(data)
        y_pred=classifier(map_s)
        loss1=loss_fn1(y_pred,labels)
        map_t = encoder(X_t)
        
        loss2 = 0
        loss3 = 0
        means_s = []
        # means_t = []
        for num in range(10):
#             subset = map_t[Y_t == num]
#             means_t.append(torch.mean(subset, dim = 0))
            subset = map_s[labels == num]
            means_s.append(torch.mean(subset, dim = 0))
        for ctr in range(10*n_support):
            num = Y_t[ctr]
            tmp = map_s[labels == num]
            leng = len(tmp)
            dd = torch.stack([ map_t[ctr] ]*leng) - tmp
            for k in range(10):
                if k == num: 
                    continue
                Cplane = torch.stack( [means_s[k]]*leng ) - tmp
                loss2 += loss_fn2(-dd, Cplane, torch.FloatTensor([-1]*leng).to(device))
                loss2 += loss_fn2(dd, Cplane, torch.FloatTensor([-1]*leng).to(device))
            
            Cplane = torch.stack( [( map_t[ctr] - means_s[i] ) for i in range(10) if i != num] )
            loss3 += loss_fn3(torch.stack([ map_t[ctr] - means_s[num] ]*9), Cplane, torch.FloatTensor([-1]*9).to(device))
            
        loss = loss1 + gamma*loss2/(n_support*90) + theta*loss3/(10*n_support)
        loss.backward()
        optimizer.step()
        
    if epoch%5 == 0: print("loss1:", loss1.item(), "   loss2:", loss2.item(), "   loss3:", loss3.item())
        
    if epoch%20 == 0:
        acc=0
        for data,labels in test_dataloader:
            data=data.to(device)
            labels=labels.to(device)
            y_test_pred=classifier(encoder(data))
            acc+=(torch.max(y_test_pred,1)[1]==labels).float().mean().item()
        accuracy=round(acc / float(len(test_dataloader)), 3)
        print("On source domain: Epoch %d/%d  accuracy: %.3f "%(epoch+1,n_epoch,accuracy))
    
    if epoch%20 == 0:
        mapset_f = []
        labelset_f = []
        for data, labels in train_dataloader:
            data = data.to(device)
            labels=labels.to(device)
            map_f = encoder(data)
            mapset_f.append(map_f)
            labelset_f.append(labels)
        map_f = torch.cat(mapset_f[:-1])
        label_f = torch.cat(labelset_f[:-1])

        means_f = []
        for num in range(10):
            subset = map_f[label_f == num]
            means_f.append(torch.mean(subset, dim = 0))
#        nume = 0
        deno = 0
        acc = 0
        for data, labels in real_test_loader:
            data = data.to(device)
            labels = labels.to(device)
            map_ff = encoder(data)
            distTS = []
            for ii in range(10):
                distTS.append(torch.norm((map_ff - means_f[ii]), dim=1))
            distTS = torch.stack(distTS)
            acc+=torch.sum(torch.argmin(distTS, dim=0)==labels)

#             for ctr in range(batch_size_test):
#                 others = []
#                 for j in range(10):
#                     num = j
#                     tmp = map_ff[ctr] - means_f[num]
#                     dd_f = torch.stack([tmp]*9)
#                     tmp = [(means_f[num] - means_f[i]) for i in range(10) if i != num]
#                     Cplane_f = torch.stack(tmp)
#                     loss_f = loss_fn2(-dd_f, Cplane_f, torch.FloatTensor([-1]*9).to(device)) + loss_fn2(dd_f, Cplane_f, torch.FloatTensor([-1]*9).to(device))
#                     others.append(loss_f.item())
#                 # print(min(others))
#                 if np.argmin(others) == labels[ctr]: 
#                     nume+=1
            deno += batch_size_test
            if deno > 5100: break
        print("-------------------------------------------------")
#        print("On target domain: Epoch %d/%d  accuracy:"%(epoch+1,n_epoch), nume / deno)
        print("Another one on TD: Epoch %d/%d  accuracy:"%(epoch+1,n_epoch), acc.item() / deno)
        print("-------------------------------------------------")
        
        

loss1: 1.5265402793884277    loss2: 354.2460021972656    loss3: 0.42317718267440796
On source domain: Epoch 1/161  accuracy: 0.915 
-------------------------------------------------
Another one on TD: Epoch 1/161  accuracy: 0.2140625
-------------------------------------------------
loss1: 1.493272304534912    loss2: 241.23873901367188    loss3: 2.2461657524108887
loss1: 1.4715677499771118    loss2: 230.93429565429688    loss3: 1.6648293733596802
loss1: 1.492607593536377    loss2: 223.9247283935547    loss3: 2.2680931091308594
On source domain: Epoch 21/161  accuracy: 0.990 
-------------------------------------------------
Another one on TD: Epoch 21/161  accuracy: 0.2388671875
-------------------------------------------------
loss1: 1.4611541032791138    loss2: 204.8499298095703    loss3: 1.7722448110580444
loss1: 1.4611506462097168    loss2: 214.4391326904297    loss3: 1.4755756855010986
loss1: 1.4612336158752441    loss2: 179.2998809814453    loss3: 0.977273166179657
loss1: 1.46142

KeyboardInterrupt: 

In [ ]:
# result check
mapset = []
labelset = []
for data, labels in train_dataloader:
    data=data.to(device)
    fmap = encoder(data).cpu().detach().numpy()
    labels=labels.to(device).cpu().detach().numpy()
    mapset.append(fmap)
    labelset.append(labels)

smap = np.vstack(mapset[:-1])
slabel = np.hstack(labelset[:-1])

means = []
dmeans = []

for num in range(10):
    subset1 = smap[slabel == num]
    means1 = np.mean(subset1, axis=0)
    tmp = subset1 - means1
    dists1 = np.linalg.norm(tmp, axis=1)
    means.append(means1)
    dmeans.append(np.mean(dists1))
print(means[0])

for i in range(10):
    for j in range(i+1):
        print(np.linalg.norm(means[i] - means[j]), end=',')
    print()

In [7]:
tmapset = []
tlabelset = []
for data, labels in real_test_loader:
    data=data.to(device)
    fmap = encoder(data).cpu().detach().numpy()
    labels=labels.to(device).cpu().detach().numpy()
    tmapset.append(fmap)
    tlabelset.append(labels)

tmap = np.vstack(tmapset[:-1])
tlabel = np.hstack(tlabelset[:-1])

tmeans = []
tdmeans = []
for num in range(10):
    subset1 = tmap[tlabel == num]
    means1 = np.mean(subset1, axis=0)
    tmp = subset1 - means1
    dists1 = np.linalg.norm(tmp, axis=1)
    tmeans.append(means1)
    tdmeans.append(np.mean(dists1))
print(dmeans)

[52.963047, 51.995987, 68.22922, 49.21099, 66.8082, 51.826233, 60.485817, 47.8431, 64.16736, 48.692932]


In [8]:
for num in range(10):
    subset1 = tmap[tlabel == num]
    tsd = []
    for i in range(10):
        tmp = np.linalg.norm((subset1 - means[i]), axis=1)
        tsd.append(np.mean(tmp))
    print(tsd)

[445.79623, 505.58197, 498.05133, 483.19223, 498.62424, 489.73657, 513.93646, 466.88065, 504.06302, 465.8715]
[489.79813, 477.8485, 517.3663, 541.3828, 500.82816, 517.25037, 545.397, 489.89743, 493.09137, 496.57635]
[479.31104, 499.38995, 453.2038, 475.05606, 469.94528, 482.3067, 512.63184, 429.37158, 453.6323, 437.70984]
[492.94138, 535.57776, 513.2114, 474.6914, 503.96408, 469.5211, 519.36365, 479.44348, 471.22406, 449.95145]
[474.0837, 501.34534, 490.56793, 495.94513, 450.35468, 471.11, 486.37878, 477.05356, 462.7748, 443.85306]
[466.91852, 511.1242, 510.3899, 456.1864, 459.6347, 386.33572, 480.76993, 458.69843, 425.5686, 413.47714]
[450.6405, 501.76154, 472.65945, 450.88358, 459.22675, 431.38123, 459.27383, 457.92798, 442.16064, 436.3949]
[463.87222, 476.37033, 472.42426, 481.8027, 481.32196, 483.51425, 543.73456, 397.78662, 464.65823, 432.1347]
[493.98096, 516.72516, 500.42645, 486.48898, 480.67206, 458.1011, 498.9799, 485.12332, 443.25677, 451.43924]
[482.16678, 524.95776, 522.28

In [9]:
print(tmeans[0])

[ -31.16322     86.0356      13.188254   -29.734535   -60.355446
   17.576384   134.30591     79.11561     37.532696   -11.958997
 -123.304985     3.1236432   25.998098   -17.795744     6.428827
   48.07155     16.341524   -58.37026     27.04379     46.83075
    7.5277233  -83.45277    -41.75638     66.62036     10.13417
   20.246445    -0.355838  -125.27577     66.08496      4.564584
   13.589962    46.206104    -3.8616078   33.55046    -59.034954
   39.694664   116.03252    -82.125694   -13.166707   -11.973564
  -43.58825    -41.531235    23.7037      51.590103   -29.155138
   36.005238   -23.759588   -20.057531   -15.161339    10.573125
  -74.27094    -15.096857   -83.16608    -35.028885   -89.51028
  -24.386696   -25.232468   129.88492     38.813408    37.56294
   15.895474   -26.870974  -179.83022     71.17689  ]
